In [1]:
import numpy as np
import scipy.stats as st
import math
import sympy as smp
import random as rd
import matplotlib.pyplot as plt

In [2]:
class def_event:
    def __init__(self,E,t):
        self.evnt = E
        self.time = t

class def_serial_episode:
    def __init__(self, evnts_ordr):
        self.evnt = evnts_ordr[:]
#         self.length = len(evnts_ordr)
        self.freq = 0

class def_window:
    def __init__(self, st_time, end_time):
        self.ts = st_time
        self.te = end_time

In [3]:
class def_episode:
    def __init__(self, evnts, edge_set):#, lastevnt, pred, succ):
        self.freq = 0
        self.evnts = evnts.copy()
        self.edges = edge_set.copy()
#         self.lastevnt = lastevnt
#         self.pred = pred.copy()
#         self.succ = succ.copy()

class def_NFA:
    def __init__(self):
        self.s = set()
        self.S = list([self.s])
        self.F = -1
        self.D = {}

class def_DFA:
    def __init__(self):
        self.s = {0}
        self.S = list([self.s])
        self.F = list()
        self.D = {}

In [4]:
def MO_Span_Recursive(ep_p,ep_p_MO_list,Tx,FT):
    global F1
    F = list()
    closed = 1
    if len(ep_p.evnt) <= 3:
        for E in F1:
            if E not in ep_p.evnt:
                evnts_temp = ep_p.evnt.copy()
                evnts_temp.append(E)
                ep = def_serial_episode(evnts_temp)
                ep_MO_list = Compute_MO_Span_List(ep_p_MO_list,E)
                ep.freq = len(ep_MO_list)
                if ep.freq == ep_p.freq:
                    closed = 0
                if ep.freq >= FT:
                    F += MO_Span_Recursive(ep,ep_MO_list,Tx,FT)
#     if closed == 1:
    F.append(ep_p)
    return(F)

In [5]:
def Compute_MO_Span_List(ep_p_MO_list,E):
    p_size = len(ep_p_MO_list)
    E_MO_list = MO_list[evnt_id[E]]
    E_size = len(E_MO_list)
    ep_MO_list = list()
    p1=0
    p2=0
    while (p1<p_size and p2<E_size):
        while p2 < E_size and E_MO_list[p2].ts <= ep_p_MO_list[p1].te :
            p2 += 1
        if p2 < E_size and E_MO_list[p2].te-ep_p_MO_list[p1].ts < Tx:
            while(p1 < p_size and ep_p_MO_list[p1].te < E_MO_list[p2].ts):
                p1 += 1
            ep_MO_list.append(def_window(ep_p_MO_list[p1-1].ts,E_MO_list[p2].te))
        else:
            p1 += 1
    return(ep_MO_list)

In [6]:
def Find_Closed(Freq_ep):
    hasher = list()
    hashtable = list()
    for ep in Freq_ep:
        fn = ep.freq
        flg = 1
        for k in range(len(hasher)):
            if fn == hasher[k]:
                flg = 0
                hashtable[k].append(ep)
                break
        if flg == 1:
            hasher.append(fn)
            hashtable.append(list([ep]))

    Freq_closed_ep = list()
    for k in range(len(hasher)):
        len_hash_k = len(hashtable[k])
        p = 0
        while p < len_hash_k:
            q = 0
            if q == p:
                q += 1
            while q < len_hash_k:
                len_p = len(hashtable[k][p].evnt)
                len_q = len(hashtable[k][q].evnt)
                if len_p < len_q:
                    sub_ep = 0
                    pi = 0
                    qi = 0
                    while(1):
                        if hashtable[k][p].evnt[pi] == hashtable[k][q].evnt[qi]:
                            pi += 1
                            if pi == len_p:
                                sub_ep = 1
                                break
                        qi += 1
                        if qi == len_q:
                            break
                    if sub_ep == 1:
                        hashtable[k].remove(hashtable[k][p])
                        len_hash_k = len(hashtable[k])
                        q -= 1
                        break
                q += 1
                if q == p:
                    q += 1
            if q >= len_hash_k:
                p += 1

    for k in range(len(hasher)):
        for l in range(len(hashtable[k])):
            Freq_closed_ep.append(hashtable[k][l])

    return Freq_closed_ep

In [7]:
def Construct_NFA(alpha):
    alpha_size = len(alpha.evnts)
    pi = {}
    for e in alpha.evnts:
        pi[e] = set()
    for (e1,e2) in alpha.edges:
        pi[e2].add(e1)
    NFA = def_NFA()
    n_states = 1
    for Q in NFA.S:
        NFA.D[str(Q)] = {}
        if len(Q) != alpha_size:
            for ev in alpha.evnts-Q:
                if not (alpha.evnts-Q).intersection(pi[ev]):
                    Q_new = Q.copy()
                    Q_new.add(ev)
                    if Q_new in NFA.S:
                        NFA.D[str(Q)][ev] = {NFA.S.index(Q_new)}
                    else:
                        NFA.S.append(Q_new)
                        NFA.D[str(Q)][ev] = {len(NFA.S)-1}
#                     NFA.D[str(Q)][ev] = [Q_new]
#                     if Q_new not in NFA.S:
#                         NFA.S.append(Q_new)
            NFA.D[str(Q)]['def'] = {NFA.S.index(Q)}
        else:
            NFA.D[str(Q)]['def'] = set()
            NFA.F = NFA.S.index(Q)
    for ev in alpha.evnts:
        if not alpha.evnts.intersection(pi[ev]):
            NFA.D[str(NFA.s)][ev].add(0)
    return NFA

In [8]:
def Construct_DFA(NFA):
    DFA = def_DFA()
    for state in DFA.S:
        if (NFA.F in state):
            DFA.F.append(state)
        DFA.D[str(state)] = {}
        W = set({})
        for substate in state:
            Q = str(NFA.S[substate])
            W = W.union(set(NFA.D[Q].keys()))
        for ev in W:
            new_state = set()
            for substate in state:
                Q = str(NFA.S[substate])
                if ev in NFA.D[Q].keys():
                    new_state = new_state.union(NFA.D[Q][ev])
#                     DFA.D[str(state)][ev] = DFA.D[str(state)][ev].union(NFA.D[Q][ev])
                else:
                    new_state = new_state.union(NFA.D[Q]['def'])
#                     DFA.D[str(state)][ev] = DFA.D[str(state)][ev].union(NFA.D[Q]['def'])
            DFA.D[str(state)][ev] = new_state.copy()
            if new_state not in DFA.S:
                        DFA.S.append(new_state)
        keys = list(DFA.D[str(state)].keys())
        for k in range(len(keys)):
            if keys[k] != 'def' and DFA.D[str(state)][keys[k]] == DFA.D[str(state)]['def']:
                del DFA.D[str(state)][keys[k]]
            else:
                k += 1
    return DFA

In [9]:
def GetTransitionMatrix(DFA,prob):
    T0 = list()
    for state in DFA.S:
        T0.append([0]*len(DFA.S))
        for ev in DFA.D[str(state)].keys():
            if ev == 'def':
                continue
            T0[-1][DFA.S.index(DFA.D[str(state)][ev])] += prob[ev]
        T0[-1][DFA.S.index(DFA.D[str(state)]['def'])] = 1 - sum(T0[-1])
    return(T0)

In [10]:
def GetStats(T0,final_states):
    
    u = smp.symbols('u')
    Dphi = smp.eye(len(T0))
    for final in final_states:
        Dphi[final,final] = u

    Tu = smp.ImmutableSparseMatrix(T0)*smp.ImmutableSparseMatrix(Dphi)
#     smp.pprint(T0)
#     smp.pprint(Dphi)
#     smp.pprint(Tu)

    lamda = smp.symbols('lamda')
#     Qn = (smp.ImmutableMatrix(lamda*smp.eye(len(T0)))-Tu).det()
    
    Qn = Tu.charpoly(lamda)
#     print(Qn)
#     print(Qn.as_expr())

    dQdu = smp.diff(Qn.as_expr(),u).subs([(u,1),(lamda,1)])
    dQdl = smp.diff(Qn.as_expr(),lamda).subs([(u,1),(lamda,1)])
    d2Qdu2 = smp.diff(smp.diff(Qn.as_expr(),u),u).subs([(u,1),(lamda,1)])
    d2Qdl2 = smp.diff(smp.diff(Qn.as_expr(),lamda),lamda).subs([(u,1),(lamda,1)])
    d2Qdudl = smp.diff(smp.diff(Qn.as_expr(),u),lamda).subs([(u,1),(lamda,1)])

    lambda1 = -dQdu/dQdl
    lambda11 = -(d2Qdu2 + 2*lambda1*d2Qdudl + lambda1**2*d2Qdl2)/dQdl
    # print(lambda1,lambda11)

    MX1 = lambda1
    VX1 = lambda11 + lambda1 - lambda1**2

#     print(MX1,VX1)
#     MX1 = 0
#     VX1 = 1
    return(MX1, VX1)

In [11]:
# ISE Length2

evs = list(['1','2'])
ISE2 = def_episode(set(evs),set({('1','2')}))

NFA = Construct_NFA(ISE2)
DFA = Construct_DFA(NFA)

final_states = set()
for k in range(len(DFA.S)):
    if (NFA.F in DFA.S[k]):
        final_states.add(k)

Prob_syms2 = {}
probs2 = smp.symbols('pe:'+str(len(evs)))
for k in range(len(evs)):
    Prob_syms2[evs[k]] = probs2[k]


T0 = GetTransitionMatrix(DFA,Prob_syms2)
MX1_2, VX1_2 = GetStats(T0,final_states)

In [12]:
# ISE Length3

evs = list(['1','2','3'])
ISE3 = def_episode(set(evs),set({('1','2'),('1','3'),('2','3')}))

NFA = Construct_NFA(ISE3)
DFA = Construct_DFA(NFA)

final_states = set()
for k in range(len(DFA.S)):
    if (NFA.F in DFA.S[k]):
        final_states.add(k)

Prob_syms3 = {}
probs3 = smp.symbols('pe:'+str(len(evs)))
for k in range(len(evs)):
    Prob_syms3[evs[k]] = probs3[k]

# Prob_values = {}
# ppp = [0.035, 0.035, 0.035]
# for k in range(len(evs)):
#     Prob_values[evs[k]] = ppp[k]

T0 = GetTransitionMatrix(DFA,Prob_syms3)
MX1_3, VX1_3 = GetStats(T0,final_states)



# mmm = MX1_3.subs([(probs[k],Prob_values[evs[k]]) for k in range(len(evs))])
# vvv = VX1_3.subs([(probs[k],Prob_values[evs[k]]) for k in range(len(evs))])

# T0 = GetTransitionMatrix(DFA,Prob_values)
# MX1_3, VX1_3 = GetStats(T0,final_states)


In [13]:
# ISE Length4

evs = list(['1','2','3','4'])
ISE4 = def_episode(set(evs),set({('1','2'),('1','3'),('1','4'),('2','3'),('2','4'),('3','4')}))

NFA = Construct_NFA(ISE4)
DFA = Construct_DFA(NFA)

final_states = set()
for k in range(len(DFA.S)):
    if (NFA.F in DFA.S[k]):
        final_states.add(k)

Prob_syms4 = {}
probs4 = smp.symbols('pe:'+str(len(evs)))
for k in range(len(evs)):
    Prob_syms4[evs[k]] = probs4[k]

# Prob_values = {}
# ppp = [0.035, 0.035, 0.035, 0.035]
# for k in range(len(evs)):
#     Prob_values[evs[k]] = ppp[k]

# T0 = GetTransitionMatrix(DFA,Prob_values)
# for row in T0:
#     print(row)
# MX1_4, VX1_4 = GetStats(T0,final_states)

T0 = GetTransitionMatrix(DFA,Prob_syms4)
# for row in T0:
#     smp.pprint(row)
MX1_4, VX1_4 = GetStats(T0,final_states)

# mmm = MX1_4.subs([(probs[k],Prob_values[evs[k]]) for k in range(len(evs))])
# vvv = VX1_4.subs([(probs[k],Prob_values[evs[k]]) for k in range(len(evs))])

In [98]:
evnt_strm = list()
# print('Enter the link of the txt file containing the event stream (for example: ./Data/Datastream_M_50_n_10000_N_4_emb_2_q1_1_q2_0.txt)')
# name = input()
name = './Data/Datastream_M_50_n_10000_N_4_emb_2_q1_1_q2_0.txt'
with open(name,'r') as f:
    for line in f:
        entry = line.split(',')
        evnt_strm.append(def_event(entry[0],int(entry[1][:-1])))

M = len(evnt_strm)
alph = list()
for m in range(len(evnt_strm)):
    A = evnt_strm[m].evnt
    flg = 1
    for a in alph:
        if a == A:
            flg = 0
            break
    if flg == 1:
        alph.append(A)
LA = len(alph) 

A = alph[0]
evnt_id = {A: 0}
count = 1
for A in alph[1:]:
    evnt_id[A] = count
    count += 1

MO_list = list()
for i in range(LA):
    MO_list.append(list())

Freq_ep = list()
F1 = alph
F1_ep = list()
for A in alph:
    F1_ep.append(def_serial_episode(list([A])))



Tx = 75
FT = 70
# print('Enter the Expiry time "Tx":')
# Tx = float(input())
# print('Enter the frequency threshold "FT":')
# FT = float(input())

for event in evnt_strm:
    E = event.evnt
    t = event.time
    F1_ep[evnt_id[E]].freq += 1
    MO_list[evnt_id[E]].append(def_window(t,t))


for E in F1:
    if F1_ep[evnt_id[E]].freq >= FT:
        Freq_ep += MO_Span_Recursive(F1_ep[evnt_id[E]],MO_list[evnt_id[E]],Tx,FT)

# print('The following are the frequent episodes discovered')
# for ep in Freq_ep:
#     print(ep.evnt ,':', ep.freq)
    
Freq_closed_ep = Find_Closed(Freq_ep)

print('Total closed frequent episodes: ', len(Freq_closed_ep))
count_FE_2 = 0
count_FE_3 = 0
count_FE_4 = 0
for k in range(len(Freq_closed_ep)):
    if len(Freq_closed_ep[k].evnt) == 2:
        count_FE_2 += 1
    if len(Freq_closed_ep[k].evnt) == 3:
        count_FE_3 += 1
    if len(Freq_closed_ep[k].evnt) == 4:
        count_FE_4 += 1
print(count_FE_2)
print(count_FE_3)
print(count_FE_4)

Enter the link of the txt file containing the event stream (for example: ./Data/Datastream_M_50_n_10000_N_4_emb_2_q1_1_q2_0.txt)



FileNotFoundError: [Errno 2] No such file or directory: ''

In [93]:
serial_episodes = list()
for ep in Freq_closed_ep:
    if len(ep.evnt) >=2 and len(ep.evnt) <=4:
        serial_episodes.append(ep)

n = 10000
Prob_Ep_Events = {}
for E in F1:
    Prob_Ep_Events[E] = len(MO_list[evnt_id[E]])/n

mean_estimate = list()
var_estimate = list()

for episode in serial_episodes:
    if len(episode.evnt) == 2:
        mmm = MX1_2.subs([(probs2[k],Prob_Ep_Events[episode.evnt[k]]) for k in range(len(episode.evnt))])
        vvv = VX1_2.subs([(probs2[k],Prob_Ep_Events[episode.evnt[k]]) for k in range(len(episode.evnt))])
        mean_estimate.append(mmm)
        var_estimate.append(vvv)
    if len(episode.evnt) == 3:
        mmm = MX1_3.subs([(probs3[k],Prob_Ep_Events[episode.evnt[k]]) for k in range(len(episode.evnt))])
        vvv = VX1_3.subs([(probs3[k],Prob_Ep_Events[episode.evnt[k]]) for k in range(len(episode.evnt))])
        mean_estimate.append(mmm)
        var_estimate.append(vvv)
    if len(episode.evnt) == 4:
        mmm = MX1_4.subs([(probs4[k],Prob_Ep_Events[episode.evnt[k]]) for k in range(len(episode.evnt))])
        vvv = VX1_4.subs([(probs4[k],Prob_Ep_Events[episode.evnt[k]]) for k in range(len(episode.evnt))])
        mean_estimate.append(mmm)
        var_estimate.append(vvv)
    print(mmm,vvv)

0.0125632520325203 0.00727256139324132
0.0134823092323122 0.00413736769324643
0.0125535947579099 0.00393745196906097
0.0129936862896735 0.00406006455712344
0.0128205061232360 0.00393599607327683
0.0135024807371867 0.00415546901817596
0.0135729674077329 0.00417096713153495
0.0128769983686786 0.00730116213040450
0.0135729674077329 0.00417096713153496
0.0125324090121317 0.00693128272271104
0.0132925187965763 0.00416848703022904
0.0138218055369932 0.00425220462519019
0.0132102985074627 0.00782647348629087
0.0123105740491674 0.00390822554191788
0.0127243570347958 0.00766352910570743
0.0125041095890411 0.00758815997961814
0.0132925187965763 0.00416848703022904
0.0134250731555700 0.00415286211292326
0.0128205061232360 0.00393599607327682
0.0126425087108014 0.00691358068420501
0.0132119426751592 0.00747829714235041
0.0129936862896735 0.00406006455712344
0.0134129419372150 0.00412186209629211
0.0128919148163556 0.00404412394252726
0.0126325200510387 0.00393861625252362
0.0132925187965763 0.0041

0.0134250731555700 0.00415286211292327
0.0116804451510334 0.00720592653569057
0.0125324090121317 0.00693128272271104
0.0123491419656786 0.00743840954174475
0.0132925187965763 0.00416848703022903
0.0131852844149392 0.00415328059800643
0.0128344615384615 0.00760132573602549
0.0134129419372150 0.00412186209629210
0.0132925187965763 0.00416848703022905
0.0127277777777778 0.00756589480986130
0.0128217845714595 0.00403759384306152
0.0130983206106870 0.00768809803319955
0.0125078956470828 0.00389783361104989
0.0131122273516703 0.00412826164889081
0.0119061611374408 0.00728552064037515
0.0131081849962914 0.00403819972660717
0.0134250731555700 0.00415286211292327
0.0124888108746424 0.00391336711575953
0.0117727323435927 0.00369768371100966
0.0124146676295137 0.00388067225734339
0.0115589836660617 0.00657564204207845
0.0121601222964383 0.00379360442882574
0.0124146676295137 0.00388067225734337
0.0120554699537750 0.00743141893213284
0.0111345110305434 0.00487669000401357
0.0115589836660617 0.0065

0.0156590278476929 0.00584633884354691
0.0167735686688089 0.00607831662784425
0.0128205061232360 0.00393599607327683
0.0131852844149392 0.00415328059800646
0.0138218055369932 0.00425220462519018
0.0138218055369932 0.00425220462519017
0.0135729674077329 0.00417096713153497
0.0122489331592544 0.00381126180047238
0.0138218055369932 0.00425220462519018
0.0128217845714595 0.00403759384306153
0.0139043218719223 0.00427558781542415
0.0129656500802568 0.00740081384404245
0.0128116492536249 0.00401022661331388
0.0134250731555700 0.00415286211292325
0.0132181013195473 0.00414372159720423
0.0137482464100318 0.00423695780056967
0.0132547112462006 0.00773898222795137
0.0131122273516703 0.00412826164889081
0.0123120828772719 0.00381912896198822
0.0114553872053872 0.00690578097180802
0.0132925187965763 0.00416848703022905
0.0131081849962914 0.00403819972660718
0.0135024807371867 0.00415546901817597
0.0131928059536818 0.00407142979888150
0.0134129419372150 0.00412186209629211
0.0126131480614005 0.0039

0.0129936862896735 0.00406006455712347
0.0128159677419355 0.00735336293334312
0.0120630283232147 0.00373442715936238
0.0119396351575456 0.00706889162324005
0.0116582127014933 0.00530958535540438
0.0128116492536249 0.00401022661331389
0.0128205061232360 0.00393599607327681
0.0128116492536249 0.00401022661331390
0.0124726060022430 0.00389788162101393
0.0125476177175566 0.00393072279113947
0.0128919148163556 0.00404412394252727
0.0128116492536249 0.00401022661331389
0.0113023584905660 0.00715753004320261
0.0126131480614005 0.00395485643213933
0.0131200099883015 0.00404790230170457
0.0130365450795058 0.00401485771708814
0.0125728284294709 0.00392155786227279
0.0129223727393694 0.00405418263765257
0.0119512380952381 0.00636718345050254
0.0132181013195473 0.00414372159720422
0.0128592884122494 0.00403680969760789
0.0114284023668639 0.00614558962096076
0.0131852844149392 0.00415328059800644
0.0115373437500000 0.00724453654901734
0.0129936862896735 0.00406006455712346
0.0132181013195473 0.0041

0.0156456594108467 0.00569236964690000
0.0145663066060190 0.00537341700588887
0.0142693594639019 0.00522413482290491
0.0135729674077329 0.00417096713153494
0.0142553472072041 0.00435526948630420
0.0131852844149392 0.00415328059800645
0.0131199029126214 0.00737716094896758
0.0135729674077329 0.00417096713153494
0.0133693907875186 0.00787889356443164
0.0128217845714595 0.00403759384306150
0.0135024807371867 0.00415546901817598
0.0127598034811985 0.00401995923566719
0.0126325200510387 0.00393861625252361
0.0139043218719223 0.00427558781542412
0.0135122171945701 0.00782194446572547
0.0134823092323122 0.00413736769324643
0.0130365450795058 0.00401485771708813
0.0122489331592544 0.00381126180047238
0.0121601222964383 0.00379360442882573
0.0132547112462006 0.00773898222795137
0.0134129419372150 0.00412186209629211
0.0128205061232360 0.00393599607327682
0.0122780067417585 0.00381881728141470
0.0139043218719223 0.00427558781542414
0.0143532411707574 0.00525587489626542
0.0153509324862475 0.0056

0.0134823092323122 0.00413736769324643
0.0139043218719223 0.00427558781542412
0.0139043218719223 0.00427558781542414
0.0135024807371867 0.00415546901817598
0.0130365450795058 0.00401485771708813
0.0137482464100318 0.00423695780056968
0.0136705660490188 0.00502109962390032
0.0136705660490188 0.00502109962390032
0.0134250731555700 0.00415286211292326
0.0132925187965763 0.00416848703022905
0.0131081849962914 0.00403819972660716
0.0124784675791904 0.00390448907476960
0.0134610271903323 0.00780553026302747
0.0169005226967386 0.00607440459958119
0.0173391182974675 0.00622496735245453
0.0181053645116919 0.00875956237558977
0.0127099215293538 0.00397171662972889
0.0131928059536818 0.00407142979888151
0.0131852844149392 0.00415328059800644
0.0135024807371867 0.00415546901817598
0.0125078956470828 0.00389783361104987
0.0124888108746424 0.00391336711575953
0.0135024807371867 0.00415546901817596
0.0128919148163556 0.00404412394252727
0.0122489331592544 0.00381126180047237
0.0131081849962914 0.0040

0.0134250731555700 0.00415286211292326
0.0138297438985189 0.00426049900219066
0.0131876855947656 0.00479376790357500
0.0135870703504421 0.00497885016780478
0.0161070549511519 0.00583045467475302
0.0163654842210063 0.00592394499980780
0.0167735686688089 0.00607831662784425
0.0159857142515864 0.00578640862198552
0.0155115165774800 0.00562871715881999
0.0151790127286537 0.00561893413514112
0.0160936488236242 0.00581549402304498
0.0153071946683874 0.00568442500606456
0.0160936488236242 0.00581549402304498
0.0227914567360351 0.0108930248525207
0.0227914567360351 0.0108930248525207
0.0212748837209302 0.0102962007798911
0.0156197964886489 0.00563478783097433
0.0160936488236242 0.00581549402304498
0.0156197964886489 0.00563478783097433
0.0144019663621721 0.00525728863889451
0.0142693594639019 0.00522413482290490
0.0144875675133872 0.00528876761872925
0.0142693594639019 0.00522413482290489
0.0138142053334802 0.00502307050979042
0.0144019663621721 0.00525728863889451
0.0127598034811985 0.0040199

0.0127598034811985 0.00401995923566718
0.0142693594639019 0.00522413482290491
0.0132575418017610 0.00482420253769293
0.0143532411707574 0.00525587489626542
0.0135729674077329 0.00417096713153494
0.0146824513550711 0.00543606273584298
0.0145663066060190 0.00537341700588888
0.0167735686688089 0.00607831662784425
0.0169005226967386 0.00607440459958120
0.0167735686688089 0.00607831662784425
0.0174845594386452 0.00626644877266842
0.0181053645116919 0.00875956237558977
0.0194734274711168 0.00935828555903943
0.0194734274711168 0.00935828555903942
0.0195591078066915 0.00969554047915991
0.0148935229591837 0.00545047671983878
0.0147052550678112 0.00540664766804566
0.0149155580990480 0.00550188329749683
0.0161295668347339 0.00585087669917569
0.0156456594108467 0.00569236964690000
0.0191921794871795 0.00937923067042642
0.0173391182974675 0.00622496735245453
0.0166553461744341 0.00605037977261125
0.0182579019073569 0.00884141789917652
0.0166553461744341 0.00605037977261123
0.0187476377952756 0.0091

In [96]:
SSTh = list()
c = 0.0001
for k in range(len(serial_episodes)):
    value = mean_estimate[k]*n + math.sqrt(var_estimate[k]*n)*st.norm.ppf(1-c)
    SSTh.append(value)

count = 0
for k in range(len(serial_episodes)):
    print(serial_episodes[k].evnt, ' : ', serial_episodes[k].freq, ' , ', SSTh[k])
    if serial_episodes[k].freq > SSTh[k]:
        count += 1

['1', '14']  :  135  ,  157.348037781946
['1', '3', '6', '2']  :  135  ,  158.744688952156
['1', '7', '8', '5']  :  135  ,  148.872448480099
['2', '1', '4', '5']  :  135  ,  153.633928619834
['2', '3', '1', '7']  :  135  ,  151.537247330947
['2', '5', '1', '7']  :  135  ,  158.998676368534
['2', '5', '3', '1']  :  135  ,  159.748207581426
['2', '38']  :  135  ,  160.547803615499
['3', '5', '2', '1']  :  135  ,  159.748207581426
['3', '13']  :  135  ,  156.286511041746
['5', '1', '4', '6']  :  135  ,  156.936579570623
['5', '2', '6', '3']  :  135  ,  162.469364146263
['5', '50']  :  135  ,  165.004141472463
['5', '6', '8', '4']  :  135  ,  146.355470441189
['5', '36']  :  135  ,  159.800430128713
['5', '29']  :  135  ,  157.437465315570
['6', '1', '5', '4']  :  135  ,  156.936579570623
['6', '5', '7', '3']  :  135  ,  158.217079450696
['7', '2', '3', '1']  :  135  ,  151.537247330947
['7', '17']  :  135  ,  157.347944762276
['1', '21']  :  138  ,  164.280420689010
['1', '2', '4', '5']  

['1', '6', '3', '4']  :  134  ,  149.665151395547
['1', '7', '2', '3']  :  134  ,  151.537247330947
['1', '7', '2', '5']  :  134  ,  158.998676368534
['1', '7', '6', '5']  :  134  ,  162.572388616105
['1', '8', '6', '5']  :  134  ,  155.820399561777
['21', '3']  :  134  ,  158.744362330508
['2', '3', '7', '6']  :  134  ,  153.930219583188
['3', '21']  :  134  ,  158.744362330508
['3', '4', '7', '8']  :  134  ,  134.758362604924
['3', '6', '5', '8']  :  134  ,  151.849243909224
['3', '7', '8', '6']  :  134  ,  144.507479767458
['4', '21']  :  134  ,  151.429028192804
['4', '5', '1', '3']  :  134  ,  150.537036185660
['5', '1', '3', '4']  :  134  ,  150.537036185660
['5', '6', '4', '7']  :  134  ,  152.221987466035
['5', '15']  :  134  ,  163.286598147923
['38', '2']  :  134  ,  160.547803615499
['50', '5']  :  134  ,  165.004141472463
['6', '3', '7', '8']  :  134  ,  144.507479767458
['6', '7', '1', '4']  :  134  ,  149.017636949988
['6', '8', '1', '5']  :  134  ,  155.820399561777
['7'

['3', '4', '6', '5']  :  147  ,  152.903621622436
['3', '7', '5', '6']  :  147  ,  158.217079450696
['5', '1', '6', '7']  :  147  ,  162.572388616105
['5', '2', '3', '4']  :  147  ,  149.799015501991
['5', '2', '6', '7']  :  147  ,  161.690255744794
['5', '6', '3', '2']  :  147  ,  162.469364146263
['6', '5', '7', '8']  :  147  ,  151.177770250249
['6', '7', '5', '3']  :  147  ,  158.217079450696
['17', '5']  :  147  ,  168.947607330780
['1', '2', '7']  :  205  ,  184.114845383071
['3', '5', '1']  :  205  ,  192.279058993625
['5', '1', '3']  :  205  ,  192.279058993625
['5', '6', '4']  :  205  ,  186.727202042036
['6', '7', '8']  :  205  ,  172.924793798669
['7', '1', '2']  :  205  ,  184.114845383071
['8', '6', '5']  :  205  ,  185.026382536846
['1', '2', '8', '3']  :  125  ,  145.762317339053
['1', '7', '8', '6']  :  125  ,  148.023289107454
['1', '36']  :  125  ,  155.151660758229
['1', '43']  :  125  ,  152.924127519797
['2', '4', '5', '1']  :  125  ,  153.633928619834
['2', '5', '

['6', '3', '1', '5']  :  146  ,  163.361115954777
['6', '3', '5', '1']  :  146  ,  163.361115954777
['6', '5', '1', '7']  :  146  ,  162.572388616105
['7', '1', '2', '5']  :  146  ,  158.998676368534
['7', '4', '5', '6']  :  146  ,  152.221987466035
['7', '5', '6', '3']  :  146  ,  158.217079450696
['7', '5', '6', '8']  :  146  ,  151.177770250249
['17', '1']  :  146  ,  163.757970386695
['17', '6']  :  146  ,  167.467377947417
['1', '3', '4', '7']  :  128  ,  143.357177940960
['1', '3', '5', '8']  :  128  ,  149.519501054911
['1', '37']  :  128  ,  156.253693697541
['1', '5', '4', '6']  :  128  ,  156.936579570623
['1', '6', '2', '7']  :  128  ,  158.006148486536
['1', '6', '2', '8']  :  128  ,  151.667666524025
['1', '6', '3', '2']  :  128  ,  158.744688952156
['1', '35']  :  128  ,  144.870199374627
['21', '6', '5']  :  128  ,  143.681447861276
['2', '1', '5', '4']  :  128  ,  153.633928619834
['2', '3', '5', '8']  :  128  ,  148.792728294400
['2', '5', '3', '4']  :  128  ,  149.799

['1', '6', '5', '3']  :  149  ,  163.361115954777
['2', '1', '5', '7']  :  149  ,  158.998676368534
['2', '3', '6', '7']  :  149  ,  153.930219583188
['2', '7', '5', '6']  :  149  ,  161.690255744795
['4', '1', '8']  :  149  ,  163.058506400630
['4', '8', '1']  :  149  ,  163.058506400631
['5', '6', '3', '7']  :  149  ,  158.217079450696
['6', '5', '1', '4']  :  149  ,  156.936579570623
['6', '7', '1', '3']  :  149  ,  154.715021161315
['6', '7', '8', '1']  :  149  ,  148.023289107454
['6', '17']  :  149  ,  167.467377947417
['1', '5', '2']  :  213  ,  197.990671237399
['2', '6', '5']  :  213  ,  202.733651169752
['4', '7']  :  213  ,  215.860862713766
['1', '5', '3', '4']  :  131  ,  150.537036185660
['1', '5', '7', '3']  :  131  ,  155.658269396318
['1', '6', '8', '5']  :  131  ,  155.820399561777
['2', '1', '7', '5']  :  131  ,  158.998676368534
['2', '4', '6', '7']  :  131  ,  148.297755450908
['2', '5', '7', '8']  :  131  ,  148.153127080883
['2', '7', '1', '5']  :  131  ,  158.99

['3', '5', '2', '6']  :  137  ,  162.469364146263
['3', '7', '6', '5']  :  137  ,  158.217079450696
['4', '1', '2', '5']  :  137  ,  153.633928619834
['4', '6', '1', '2']  :  137  ,  152.717398933979
['5', '21']  :  137  ,  169.503786837824
['5', '2', '1', '6']  :  137  ,  167.096922613631
['5', '3', '2', '1']  :  137  ,  159.748207581426
['7', '1', '3', '6']  :  137  ,  154.715021161315
['7', '3', '5', '2']  :  137  ,  154.861645736316
['7', '4', '6', '5']  :  137  ,  152.221987466035
['7', '8', '1', '6']  :  137  ,  148.023289107454
['8', '2', '5', '6']  :  137  ,  155.017530800795
['3', '2', '5']  :  189  ,  190.914772215173
['5', '6', '7', '1']  :  189  ,  162.572388616105
['6', '3', '7']  :  189  ,  183.017004010274
['6', '7', '4']  :  189  ,  174.398445433213
['6', '8', '1']  :  189  ,  179.667707503139
['7', '3', '1']  :  189  ,  179.223759549174
['2', '1', '3', '5']  :  148  ,  159.748207581426
['2', '1', '6', '7']  :  148  ,  158.006148486536
['2', '3', '1', '6']  :  148  ,  1

In [97]:
print(len(serial_episodes),count)
count_FE_2 = 0
count_FE_3 = 0
count_FE_4 = 0
count_SS_2 = 0
count_SS_3 = 0
count_SS_4 = 0
for k in range(len(serial_episodes)):
    if len(serial_episodes[k].evnt) == 2:
        count_FE_2 += 1
        if serial_episodes[k].freq > SSTh[k]:
            count_SS_2 += 1
    if len(serial_episodes[k].evnt) == 3:
        count_FE_3 += 1
        if serial_episodes[k].freq > SSTh[k]:
            count_SS_3 += 1
    if len(serial_episodes[k].evnt) == 4:
        count_FE_4 += 1
        if serial_episodes[k].freq > SSTh[k]:
#             print(serial_episodes[k].evnt, ' : ', serial_episodes[k].freq, ' , ', SSTh[k])
            count_SS_4 += 1
print(count_SS_2)
print(count_SS_3)
print(count_SS_4)

1541 267
29
196
42
